In [2]:
!pip install -U -q transformers datasets bitsandbytes trl peft

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

#model_name = "cognitya/llama-2-1B-instruct-edited"
model_name = "meta-llama/Llama-3.2-1B-Instruct"

config_8bit = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=config_8bit,
    device_map="auto",
    trust_remote_code=True,
)

In [5]:
model_8bit

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMS

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left", trust_remote_code=True)

In [8]:
from datasets import load_dataset
dataset = load_dataset("MattCoddity/dockerNLcommands")

README.md:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

06102023.json:   0%|          | 0.00/543k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2415 [00:00<?, ? examples/s]

In [10]:
dataset['train'][0]

{'input': 'Give me a list of containers that have the Ubuntu image as their ancestor.',
 'output': "docker ps --filter 'ancestor=ubuntu'",
 'instruction': 'translate this sentence in docker command'}

In [11]:
from datasets import DatasetDict
train_test_split = dataset['train'].train_test_split(test_size=0.2,seed=42)
dataset = DatasetDict(
    {
        'train':train_test_split['train'], 
        'validation' :train_test_split['test']
    }
)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 483
    })
})

In [ ]:
def chat_template(example):
    mesagges =[
        {'role':'system', 'content':example['instruction']},
        {'role':'user', 'content':example['input']},
        {'role':'assistant', 'content':example['output']},
    ]